# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f717c5e0f70>
├── 'a' --> tensor([[-0.6225,  1.5682,  0.7083],
│                   [ 0.0514, -1.0173, -0.7090]])
└── 'x' --> <FastTreeValue 0x7f717c5e0f40>
    └── 'c' --> tensor([[-0.2995,  0.8026,  0.9236, -0.7262],
                        [-2.7285, -0.5340,  0.3088, -0.5385],
                        [ 0.6102,  0.6896, -0.3087, -0.0344]])

In [4]:
t.a

tensor([[-0.6225,  1.5682,  0.7083],
        [ 0.0514, -1.0173, -0.7090]])

In [5]:
%timeit t.a

72.8 ns ± 1.28 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f717c5e0f70>
├── 'a' --> tensor([[-0.3065,  0.6307, -1.4207],
│                   [-0.0506, -1.0080, -1.2963]])
└── 'x' --> <FastTreeValue 0x7f717c5e0f40>
    └── 'c' --> tensor([[-0.2995,  0.8026,  0.9236, -0.7262],
                        [-2.7285, -0.5340,  0.3088, -0.5385],
                        [ 0.6102,  0.6896, -0.3087, -0.0344]])

In [7]:
%timeit t.a = new_value

72.7 ns ± 1.25 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6225,  1.5682,  0.7083],
               [ 0.0514, -1.0173, -0.7090]]),
    x: Batch(
           c: tensor([[-0.2995,  0.8026,  0.9236, -0.7262],
                      [-2.7285, -0.5340,  0.3088, -0.5385],
                      [ 0.6102,  0.6896, -0.3087, -0.0344]]),
       ),
)

In [10]:
b.a

tensor([[-0.6225,  1.5682,  0.7083],
        [ 0.0514, -1.0173, -0.7090]])

In [11]:
%timeit b.a

66.5 ns ± 0.996 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0962,  0.5093, -1.8085],
               [ 0.4205, -1.3088,  0.0386]]),
    x: Batch(
           c: tensor([[-0.2995,  0.8026,  0.9236, -0.7262],
                      [-2.7285, -0.5340,  0.3088, -0.5385],
                      [ 0.6102,  0.6896, -0.3087, -0.0344]]),
       ),
)

In [13]:
%timeit b.a = new_value

609 ns ± 15.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.03 µs ± 21.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.9 µs ± 231 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

288 µs ± 5.94 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

268 µs ± 4.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f717c5a5070>
├── 'a' --> tensor([[[-0.6225,  1.5682,  0.7083],
│                    [ 0.0514, -1.0173, -0.7090]],
│           
│                   [[-0.6225,  1.5682,  0.7083],
│                    [ 0.0514, -1.0173, -0.7090]],
│           
│                   [[-0.6225,  1.5682,  0.7083],
│                    [ 0.0514, -1.0173, -0.7090]],
│           
│                   [[-0.6225,  1.5682,  0.7083],
│                    [ 0.0514, -1.0173, -0.7090]],
│           
│                   [[-0.6225,  1.5682,  0.7083],
│                    [ 0.0514, -1.0173, -0.7090]],
│           
│                   [[-0.6225,  1.5682,  0.7083],
│                    [ 0.0514, -1.0173, -0.7090]],
│           
│                   [[-0.6225,  1.5682,  0.7083],
│                    [ 0.0514, -1.0173, -0.7090]],
│           
│                   [[-0.6225,  1.5682,  0.7083],
│                    [ 0.0514, -1.0173, -0.7090]]])
└── 'x' --> <FastTreeValue 0x7f70c6930dc0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

44 µs ± 877 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f70c693f0a0>
├── 'a' --> tensor([[-0.6225,  1.5682,  0.7083],
│                   [ 0.0514, -1.0173, -0.7090],
│                   [-0.6225,  1.5682,  0.7083],
│                   [ 0.0514, -1.0173, -0.7090],
│                   [-0.6225,  1.5682,  0.7083],
│                   [ 0.0514, -1.0173, -0.7090],
│                   [-0.6225,  1.5682,  0.7083],
│                   [ 0.0514, -1.0173, -0.7090],
│                   [-0.6225,  1.5682,  0.7083],
│                   [ 0.0514, -1.0173, -0.7090],
│                   [-0.6225,  1.5682,  0.7083],
│                   [ 0.0514, -1.0173, -0.7090],
│                   [-0.6225,  1.5682,  0.7083],
│                   [ 0.0514, -1.0173, -0.7090],
│                   [-0.6225,  1.5682,  0.7083],
│                   [ 0.0514, -1.0173, -0.7090]])
└── 'x' --> <FastTreeValue 0x7f70c693f070>
    └── 'c' --> tensor([[-0.2995,  0.8026,  0.9236, -0.7262],
                        [-2.7285, -0.5340,  0.3088, -0.5385],
                 

In [23]:
%timeit t_cat(trees)

41.8 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

79 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.2995,  0.8026,  0.9236, -0.7262],
                       [-2.7285, -0.5340,  0.3088, -0.5385],
                       [ 0.6102,  0.6896, -0.3087, -0.0344]],
              
                      [[-0.2995,  0.8026,  0.9236, -0.7262],
                       [-2.7285, -0.5340,  0.3088, -0.5385],
                       [ 0.6102,  0.6896, -0.3087, -0.0344]],
              
                      [[-0.2995,  0.8026,  0.9236, -0.7262],
                       [-2.7285, -0.5340,  0.3088, -0.5385],
                       [ 0.6102,  0.6896, -0.3087, -0.0344]],
              
                      [[-0.2995,  0.8026,  0.9236, -0.7262],
                       [-2.7285, -0.5340,  0.3088, -0.5385],
                       [ 0.6102,  0.6896, -0.3087, -0.0344]],
              
                      [[-0.2995,  0.8026,  0.9236, -0.7262],
                       [-2.7285, -0.5340,  0.3088, -0.5385],
                       [ 0.6102,  0.6896, -0.3087, -0.0344]],

In [26]:
%timeit Batch.stack(batches)

104 µs ± 1.79 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.2995,  0.8026,  0.9236, -0.7262],
                      [-2.7285, -0.5340,  0.3088, -0.5385],
                      [ 0.6102,  0.6896, -0.3087, -0.0344],
                      [-0.2995,  0.8026,  0.9236, -0.7262],
                      [-2.7285, -0.5340,  0.3088, -0.5385],
                      [ 0.6102,  0.6896, -0.3087, -0.0344],
                      [-0.2995,  0.8026,  0.9236, -0.7262],
                      [-2.7285, -0.5340,  0.3088, -0.5385],
                      [ 0.6102,  0.6896, -0.3087, -0.0344],
                      [-0.2995,  0.8026,  0.9236, -0.7262],
                      [-2.7285, -0.5340,  0.3088, -0.5385],
                      [ 0.6102,  0.6896, -0.3087, -0.0344],
                      [-0.2995,  0.8026,  0.9236, -0.7262],
                      [-2.7285, -0.5340,  0.3088, -0.5385],
                      [ 0.6102,  0.6896, -0.3087, -0.0344],
                      [-0.2995,  0.8026,  0.9236, -0.7262],
                   

In [28]:
%timeit Batch.cat(batches)

186 µs ± 1.93 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

601 µs ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
